In [1]:
!pip install konlpy

In [2]:
!pip install Twitter

In [3]:
import numpy as np
import pandas as pd
import re
import os
import sys

In [4]:
#자바 경로 설정
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"

#환경변수 path 정보에 자바설치 경로의 bin 폴더 추가
os.environ["PATH"] = "C:\\Program Files\\Java\\jdk-17\\bin;" + os.environ["PATH"]

In [5]:
from konlpy.tag import Twitter
import pickle

In [6]:
#수어 스크립트 단어 개수
MAX_SEQUENCE = 30

#스크립트의 단어가 30개 미만일 때 <PAD>로 채워서 30개로 맞춤
PAD = "<PAD>"

#스크립트 시작, 끝
STA = "<START>"
END = "<END>"

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2

In [7]:
#수어 스크립트 파일의 경로
korean_text_csv_path = "data/korean_text.csv"

#수어 스크립트 파일의 내용을 읽어서 리턴
df = pd.read_csv(korean_text_csv_path)

In [8]:
#korean_text 열(수어스크립트) 리턴
korean_text = df["korean_text"]

#korean_text 열의 내용을 리스트로 변환
temp = korean_text.values.tolist()

In [9]:
#문장에서 단어를 추출할 객체
tagger = Twitter()

for sentence in temp[0:2]:
    #sentence를 단어로 분리해서 리스트에 저장
    morphs = tagger.morphs(sentence)

    #morphs 리스트를 문자열로 변환
    sentence_str = " ".join(morphs)
    print("sentence_str=", sentence_str)
    print("=" * 100)

C:\ProgramData\anaconda3\envs\ict_sign_language_env01\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


sentence_str= 충남 보령시 동남쪽 4 km 지역 규모 3 5 지진 발생 여진 등 안전 에 주의 바랍니다
sentence_str= 경북 상주시 북쪽 5 km 지역 규모 4 3 지진 발생 여진 등 안전 에 주의 바랍니다


In [10]:
#수어스크립트 전체 리스트를 저장
def pos_tag(all_sentence):
    tagger = Twitter()

    #수어 스크립트를 단어별로 분리한 결과를 저장할 리스트
    sentence_pos = []

    for sentence in all_sentence:
        #sentence를 단어로 분리해서 리스트에 저장
        morphs = tagger.morphs(sentence)

        #morphs 리스트를 문자열로 변환
        sentence_str = " ".join(morphs)

        sentence_pos.append(sentence_str)
        
    return sentence_pos

In [11]:
#temp 문자를 단어별로 분리해서 리턴
answer_list = pos_tag(temp)

In [12]:
#df의 korean_text_split 열에 answer_list 저장
df["korean_text_split"] = answer_list

word_list = []

for answer in answer_list:
    #문장을 단어로 분리하고 단어를 word_list에 추가
    for word in answer.split():
        word_list.append(word)

#word_list의 중복 단어 제거
word_list = list(set(word_list))

#word_list의 맨 앞에 pad, sta, end 추가
word_list[:0] = [PAD, STA, END]

#word_list에서 단어는 word에, 인덱스는 index에 저장
word_to_index = {word: index for index, word in enumerate(word_list)}

index_to_word = {index: word for index, word in enumerate(word_list)}

In [13]:
row_num = len(df)

In [14]:
#df에 text_encoding 열 추가
df["text_encoding"] = None

In [15]:
df

,video_id,korean_text,korean_text_split,text_encoding
0,NIA_SL_G1_EARTHQUAKE000013_1_CROWD1079481,충남 보령시 동남쪽 4km 지역 규모 3 5 지진발생 여진 등 안전에 주의바랍니다,충남 보령시 동남쪽 4 km 지역 규모 3 5 지진 발생 여진 등 안전 에 주의 바랍니다,None
1,NIA_SL_G1_EARTHQUAKE000014_1_CROWD1079481,경북 상주시 북쪽 5km 지역 규모 4 3 지진발생 여진 등 안전에 주의바랍니다,경북 상주시 북쪽 5 km 지역 규모 4 3 지진 발생 여진 등 안전 에 주의 바랍니다,None
2,NIA_SL_G1_EARTHQUAKE000015_1_CROWD1079481,울진군 남쪽 6km 지역 규모 3 3 지진발생 여진 등 안전에 주의바랍니다,울진군 남쪽 6 km 지역 규모 3 3 지진 발생 여진 등 안전 에 주의 바랍니다,None
3,NIA_SL_G1_EARTHQUAKE000016_1_CROWD1079481,경북 경주시 남서쪽 10km 지역 규모 3 1 지진발생 여진 등 안전에 주의바랍니다,경북 경주시 남서쪽 10 km 지역 규모 3 1 지진 발생 여진 등 안전 에 주의 ...,None
4,NIA_SL_G1_EARTHQUAKE000017_1_CROWD1079481,경북 상주시 남쪽 9km 지역 규모 3 0 지진발생 여진 등 안전에 주의바랍니다,경북 상주시 남쪽 9 km 지역 규모 3 0 지진 발생 여진 등 안전 에 주의 바랍니다,None
...,...,...,...,...
347,NIA_SL_G4_SUMMERTIMEWATERGAME000096_1_TW07,행정안전부 안전사고 대비 요령 갯벌에 발이 깊이 빠진 경우 반대방향으로 엎드...,행정안전부 안전 사고 대비 요령 갯벌 에 발 이 깊이 빠진 경우 반 대 방향 으로 ...,None
348,NIA_SL_G4_SUMMERTIMEWATERGAME000097_1_TW07,행정안전부 안전사고 대비 요령 갯벌에는 맨발로 들어가지 않는다 발에 잘 맞...,행정안전부 안전 사고 대비 요령 갯벌 에는 맨발 로 들어가지 않는다 발 에 잘 맞는...,None
349,NIA_SL_G4_SUMMERTIMEWATERGAME000098_1_TW07,행정안전부 갯벌에는 어패류의 패각 등이 있어 맨발로 출입할 경우 심한 상처를 입...,행정안전부 갯벌 에는 어패류 의 패각 등 이 있어 맨발 로 출입 할 경우 심한 상처...,None
350,NIA_SL_G4_SUMMERTIMEWATERGAME000099_1_TW07,행정안전부 안전사고 대비 요령 갯벌체험시 되도록 긴팔옷을 착용하며 창이 넓은...,행정안전부 안전 사고 대비 요령 갯벌 체험 시 되도록 긴 팔 옷 을 착용 하며 창 ...,None


In [16]:
#각 행에 sta, end 추가
df["korean_text_split"] = df["korean_text_split"].apply(lambda x: STA + " " + x + " " + END)

In [17]:
df.loc[0, "korean_text_split"]

'<START> 충남 보령시 동남쪽 4 km 지역 규모 3 5 지진 발생 여진 등 안전 에 주의 바랍니다 <END>'

In [18]:
for i in range(row_num):
    index_list = []
    #i번째행 korean_text_split 열 리턴
    sentence = df.loc[i, "korean_text_split"]

    #sentence를 단어들로 분리
    for word in sentence.split():
        #word_to_index에서 단어의 인덱스를 리턴하고 index_list에 가가
        index = word_to_index.get(word)
        index_list.append(index)

    #index_list에 저장된 단어의 수가 30개 미만이면 PAD 추가
    if len(index_list) < MAX_SEQUENCE:
        print("0으로 채움")
        print(len(index_list))
        print((MAX_SEQUENCE - len(index_list)) * [word_to_index.get(PAD)])
        print("=" *100)
        index_list += (MAX_SEQUENCE - len(index_list)) * [word_to_index.get(PAD)]
        print(len(index_list))
    else:
        index_list = index_list[:MAX_SEQUENCE]    
        index_list[-1] = END_INDEX

    #df의 i번째 행 text_encoding 열에 index_list 대입
    df.at[i, "text_encoding"] = index_list

0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
18
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
15
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
15
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
19
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
30
0으로 채움
12
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [19]:
df.to_csv("data/y_train_preprocess.csv", index=False, encoding='utf-8-sig')

In [20]:
with open("data/word_to_index.pickle", "wb") as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)

In [21]:
with open("data/index_to_word.pickle", "wb") as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)